In [1]:
import jpype
import jpype.imports as jimport

In [2]:
jimport.registerDomain('org')
jpype.addClassPath("../../java/target/python-matsim-1.0-SNAPSHOT-jar-with-dependencies.jar")
jpype.startJVM(jpype.get_default_jvm_path(), "-Djava.class.path=%s" % jpype.getClassPath())

In [3]:
import sys
sys.path.append('..')
from pythonmatsim.api.events import *

In [11]:
import pythonmatsim.api.bindings as matsim
import pythonmatsim.api.logging

In [5]:
from org.matsim.core.scenario import ScenarioUtils
from org.matsim.core.config import ConfigUtils
from org.matsim.api.core.v01 import Id
from org.matsim.core.controler import Controler
from org.matsim.api.core.v01.events.handler import ActivityStartEventHandler, ActivityEndEventHandler
# Note: this does not work when using JPackage approach
from org.matsim.core.controler.OutputDirectoryHierarchy import OverwriteFileSetting


In [6]:
from java.net import URL, URI
from collections import defaultdict
import time

In [7]:
config_url = URI('https://raw.githubusercontent.com/matsim-org/matsim/master/examples/scenarios/siouxfalls-2014/config_default.xml').toURL()
#config_url = URI('https://raw.githubusercontent.com/matsim-org/matsim/master/examples/scenarios/equil/config.xml').toURL()
config = ConfigUtils.loadConfig(config_url)
config.controler().setLastIteration(1)
config.controler().setOverwriteFileSetting(OverwriteFileSetting.deleteDirectoryIfExists)


In [8]:
scenario = ScenarioUtils.loadScenario(config)
controler = Controler(scenario)

start = time.time()
controler.run()
duration_without_events = time.time() - start
duration_without_events

80.49754476547241

In [12]:
class TestListener(EventListener):
    def __init__(self):
        self.counts = defaultdict(int)

    def reset(self, iteration):
        self.iteration = iteration

    @listen_to(EventType.actStart, EventType.actEnd)
    def handleAct(self, event):
        event.time
        event.linkId
        event.facilityId
        event.persId
        event.actType
        self.counts[self.iteration] += 1

In [13]:
listener = TestListener()

In [ ]:
scenario = ScenarioUtils.loadScenario(config)
controler = Controler(scenario)
add_event_handler(
    controler,
    listener,
    10)

start = time.time()
controler.run()
duration_with_events = time.time() - start
duration_with_events

In [12]:
listener.counts


defaultdict(int, {0: 168537, 1: 173537})

In [13]:
class NaiveHandler:
    ####################################################################
    # Methods for the python side
    ####################################################################
    def __init__(self):
        self.counts = defaultdict(int)

    ####################################################################
    # Methods for the java side
    ####################################################################
        
    def handleEvent(self, event):
        event.getTime()
        event.getLinkId()
        event.getFacilityId()
        event.getPersonId()
        event.getActType()
        self.counts[self.iteration] += 1
        
    def reset(self, iteration):
        self.iteration = iteration
    
    # JProxy does not provide default implementation for those methods:
    # provide one, to avoid crashes.
    # In true python way, the JProxy checks if a method exists only if actually
    # invoked.
    def hashCode(self):
        return 42
    
    def toString(self):
        return "%s" % self

handler = NaiveHandler()
handlerProxy = jp.JProxy((ActivityStartEventHandler, ActivityEndEventHandler), inst=handler)

In [14]:
controler = Controler(scenario)

controler.getEvents().addHandler(handlerProxy)

start = time.time()
controler.run()
duration_naive = time.time() - start
duration_naive

94.36490178108215